In [2]:
import os
import time
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout

In [3]:
!pip install BeautifulSoup
!pip install playwright
!playwright install
!playwright install-deps

  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Installing dependencies...
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
SEASONS = list(range(2015, 2024))
SEASONS

[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [6]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Basketball Analytics Project/data"

In [7]:
def is_valid_integer(string):
    try:
        integer = int(string)
        return True
    except ValueError:
        return False


In [8]:
async def get_html(url, selector, sleep=5, retries=3):
  html = None
  for i in range(1, retries+1):
    time.sleep(sleep * i)

    try:
      async with async_playwright() as p:
        browser = await p.firefox.launch()
        page = await browser.new_page()
        await page.goto(url)
        print(await page.title())
        html = await page.inner_html(selector)
    except PlaywrightTimeout:
      print(f"Tiemout error on {url}")
      continue
    break

  return html

In [9]:
async def scrape_season(season):
  url = f'https://www.basketball-reference.com/players/c/curryst01/gamelog/2016'
  html = await get_html(url, "#inner_nav .hoversmooth")

  soup = BeautifulSoup(html)
  links = soup.find_all("a")
  visited = set()  # Set to store visited URLs
  games_pages = []

  for link in links:
      href = link.get("href")
      if href and "gamelog" in href and is_valid_integer(href.split('/')[-1]):
          season = int(href.split('/')[-1])
          if season in SEASONS and href not in visited:
              games_pages.append(f"https://www.basketball-reference.com{href}")
              visited.add(href)


  for url in games_pages:
    save_path = os.path.join(DATA_DIR, url.split("/")[-1])
    if os.path.exists(save_path):
      continue

    html = await get_html(url, "#content")
    if not html:
      continue
    with open(save_path, "w+") as f:
      f.write(html)


In [14]:
for season in SEASONS:
    await scrape_season(season)

Stephen Curry 2015-16 Game Log | Basketball-Reference.com
Stephen Curry 2014-15 Game Log | Basketball-Reference.com
Stephen Curry 2015-16 Game Log | Basketball-Reference.com
Stephen Curry 2016-17 Game Log | Basketball-Reference.com
Stephen Curry 2017-18 Game Log | Basketball-Reference.com
Stephen Curry 2018-19 Game Log | Basketball-Reference.com
Stephen Curry 2019-20 Game Log | Basketball-Reference.com
Stephen Curry 2020-21 Game Log | Basketball-Reference.com
Stephen Curry 2021-22 Game Log | Basketball-Reference.com
Stephen Curry 2022-23 Game Log | Basketball-Reference.com
Stephen Curry 2015-16 Game Log | Basketball-Reference.com
Stephen Curry 2015-16 Game Log | Basketball-Reference.com
Stephen Curry 2015-16 Game Log | Basketball-Reference.com
Stephen Curry 2015-16 Game Log | Basketball-Reference.com
Stephen Curry 2015-16 Game Log | Basketball-Reference.com
Stephen Curry 2015-16 Game Log | Basketball-Reference.com
Stephen Curry 2015-16 Game Log | Basketball-Reference.com
Stephen Curry 

In [12]:
# !find '/content/drive/MyDrive/Colab Notebooks/Basketball Analytics Project/data' -type f -delete